In [38]:
!gdown https://drive.google.com/uc?id=13GI1F1hvwpMUGBSa0QC6ov4eE57GC_Zx

Downloading...
From: https://drive.google.com/uc?id=13GI1F1hvwpMUGBSa0QC6ov4eE57GC_Zx
To: /workspaces/evals/evals/datasets/UNv1.0.testsets.tar.gz
100%|███████████████████████████████████████| 3.62M/3.62M [00:00<00:00, 202MB/s]


In [40]:
!tar -xzvf UNv1.0.testsets.tar.gz

testsets/
testsets/UNv1.0.pdf
testsets/testset/
testsets/testset/UNv1.0.testset.fr
testsets/testset/UNv1.0.testset.ids
testsets/testset/UNv1.0.testset.ar
testsets/testset/UNv1.0.testset.zh
testsets/testset/UNv1.0.testset.ru
testsets/testset/UNv1.0.testset.en
testsets/testset/UNv1.0.testset.es
testsets/devset/
testsets/devset/UNv1.0.devset.fr
testsets/devset/UNv1.0.devset.es
testsets/devset/UNv1.0.devset.ru
testsets/devset/UNv1.0.devset.ids
testsets/devset/UNv1.0.devset.zh
testsets/devset/UNv1.0.devset.en
testsets/devset/UNv1.0.devset.ar
testsets/README
testsets/DISCLAIMER


In [1]:
src = "ar"
trg = "en"

In [8]:
sys_msg = f"Translate the following statement from {src} to {trg}"
def create_chat_prompt(sys_msg, input_text):
    return [
        {"role": "system", "content": sys_msg}, 
        {"role": "user", "content": input_text}
    ]

def create_chat_example(article, summary):
    return [
        {"role": "system", "content": article, "name": "example_user"},
        {"role": "system", "content": summary, "name": "example_assistant"},
    ]

In [4]:
!mkdir ../registry/data/un_v1

In [5]:
import pandas as pd

src_path = "testsets/testset/UNv1.0.testset.ar"
trg_path = "testsets/testset/UNv1.0.testset.en"

df_test_tar = pd.read_csv(trg_path, delimiter="  ", header = None)
df_test_src = pd.read_csv(src_path, delimiter="  ", header = None)

df_test = pd.concat([df_test_tar, df_test_src], axis = 1)
df_test.columns = ['target', "source"]

src_path = "testsets/devset/UNv1.0.devset.ar"
trg_path = "testsets/devset/UNv1.0.devset.en"

df_dev_tar = pd.read_csv(trg_path, delimiter="  ", header = None)
df_dev_src = pd.read_csv(src_path, delimiter="  ", header = None)

df_dev = pd.concat([df_dev_tar, df_dev_src], axis = 1)
df_dev.columns = ['target', "source"]

/tmp/ipykernel_4980/2369823548.py:6: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  df_test_tar = pd.read_csv(trg_path, delimiter="  ", header = None)
/tmp/ipykernel_4980/2369823548.py:7: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  df_test_src = pd.read_csv(src_path, delimiter="  ", header = None)
/tmp/ipykernel_4980/2369823548.py:15: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  df_dev_tar = pd.read_csv(trg_pa

In [19]:
df_test = df_test[:1]

In [21]:
df_test

,target,source,input,ideal
0,"7439th meeting, held on 11 May 2015.",الجلسة 7439 المعقودة في 11 أيار/مايو ٢٠١٥.,"[{'role': 'system', 'content': 'Translate the ...","7439th meeting, held on 11 May 2015."


In [20]:
df_dev["sample"] = df_dev.apply(lambda x: create_chat_example(x['source'], x['target']), axis=1)
df_dev[["sample"]].to_json(f'../registry/data/un_v1/few_shot_{src}_{trg}.jsonl', lines=True, orient="records")

df_test["input"] = df_test['source'].apply(lambda x: create_chat_prompt(sys_msg, x))
df_test["ideal"] = df_test['target']
df_test[["input", "ideal"]].to_json(f'../registry/data/un_v1/samples_{src}_{trg}.jsonl', lines=True, orient="records")

/tmp/ipykernel_4980/1063504306.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_test["input"] = df_test['source'].apply(lambda x: create_chat_prompt(sys_msg, x))
/tmp/ipykernel_4980/1063504306.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_test["ideal"] = df_test['target']


In [22]:
import os
os.environ["OPENAI_API_KEY"] = "<openai-key>"

In [23]:
!oaieval gpt-3.5-turbo translation --max_samples 1

[2023-04-18 13:56:03,723] [registry.py:156] Loading registry from /workspaces/evals/evals/registry/evals
[2023-04-18 13:56:03,815] [registry.py:156] Loading registry from /home/codespace/.evals/evals
[2023-04-18 13:56:04,287] [oaieval.py:213] Run started: 230418135604NGTLX5AX
[2023-04-18 13:56:04,305] [data.py:75] Fetching un_v1/samples_ar_en.jsonl
[2023-04-18 13:56:04,306] [eval.py:32] Evaluating 1 samples
[2023-04-18 13:56:04,310] [eval.py:152] Running in threaded mode with 10 threads!
100%|█████████████████████████████████████████████| 1/1 [00:01<00:00,  1.51s/it]
[2023-04-18 13:56:05,824] [record.py:320] Final report: {'accuracy': 0.0, 'sacrebleu_score': 14.923729480049115}. Logged to /tmp/evallogs/230418135604NGTLX5AX_gpt-3.5-turbo_translation.jsonl
[2023-04-18 13:56:05,824] [oaieval.py:231] Final report:
[2023-04-18 13:56:05,824] [oaieval.py:233] accuracy: 0.0
[2023-04-18 13:56:05,824] [oaieval.py:233] sacrebleu_score: 14.923729480049115
[2023-04-18 13:56:05,827] [record.py:309] 

In [24]:
!ls /tmp/evallogs

2304181347523FVPCZUS_gpt-3.5-turbo_translation.jsonl
230418135604NGTLX5AX_gpt-3.5-turbo_translation.jsonl


In [25]:
import pandas as pd
events = f"/tmp/evallogs/230418135604NGTLX5AX_gpt-3.5-turbo_translation.jsonl"

with open(events, "r") as f:
    events_df = pd.read_json(f, lines=True)

In [31]:
for i, r in pd.json_normalize(events_df[events_df.type == "sampling"].data).iterrows():
    print(f"Prompt: {r.prompt[0]['content']} {r.prompt[-1]['content']}")
    print(f"Predict: {r.sampled}")
    print(f"Truth: {df_test['target'][0]}")
    print(f"--" * 25)

Prompt: Translate the following statement from ar to en الجلسة 7439 المعقودة في 11 أيار/مايو ٢٠١٥.
Predict: Session 7439 held on May 11th, 2015.
Truth: 7439th meeting, held on 11 May 2015.
--------------------------------------------------
